Импортируем необходимые библиотеки которые мы точно будем использовать

In [1]:
import pandas as pd
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics import precision_recall_curve, classification_report
from sklearn import preprocessing, datasets, metrics, tree 

In [2]:
pip install catboost

In [3]:
from catboost import  CatBoostRegressor

Загрузим данные 

In [4]:
!wget https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-train.csv
!wget https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-test.csv
!wget https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-desc.csv

--2022-05-05 21:12:55--  https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5329268 (5.1M) [text/plain]
Saving to: ‘PD-data-train.csv’

PD-data-train.csv   100%[===================>]   5.08M  --.-KB/s    in 0.06s   

2022-05-05 21:12:55 (90.4 MB/s) - ‘PD-data-train.csv’ saved [5329268/5329268]

--2022-05-05 21:12:55--  https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28259 (28

Создадим из csv DataFrame

In [5]:
train_df = pd.read_csv('PD-data-train.csv', sep=';')
test_df = pd.read_csv('PD-data-test.csv', sep=';')
desc_df = pd.read_csv('PD-data-desc.csv', sep=';')

Выведем базовую информацию об обучающей выборке

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32395 entries, 0 to 32394
Data columns (total 37 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   record_id                     32395 non-null  int64  
 1   ar_revenue                    17250 non-null  float64
 2   ar_total_expenses             17250 non-null  float64
 3   ar_sale_cost                  17250 non-null  float64
 4   ar_selling_expenses           17250 non-null  float64
 5   ar_management_expenses        17250 non-null  float64
 6   ar_sale_profit                17250 non-null  float64
 7   ar_balance_of_rvns_and_expns  17250 non-null  float64
 8   ar_profit_before_tax          17250 non-null  float64
 9   ar_taxes                      17250 non-null  float64
 10  ar_other_profit_and_losses    17250 non-null  float64
 11  ar_net_profit                 17250 non-null  float64
 12  ab_immobilized_assets         17250 non-null  float64
 13  a

Видим что колонка 'ul_staff_range' имеет тип object

In [7]:
train_df['ul_staff_range'].unique()

array(['[1-100]', '(100-500]', '> 500'], dtype=object)

Преобразуем категориальный признак 

In [8]:
train_df.loc[train_df['ul_staff_range'] == '[1-100]', 'ul_staff_range'] = 1
train_df.loc[train_df['ul_staff_range'] == '(100-500]', 'ul_staff_range'] = 2
train_df.loc[train_df['ul_staff_range'] == '> 500', 'ul_staff_range'] = 3
test_df.loc[test_df['ul_staff_range'] == '[1-100]', 'ul_staff_range'] = 1
test_df.loc[test_df['ul_staff_range'] == '(100-500]', 'ul_staff_range'] = 2
test_df.loc[test_df['ul_staff_range'] == '> 500', 'ul_staff_range'] = 3
train_df['ul_staff_range'] = pd.to_numeric(train_df['ul_staff_range'])
test_df['ul_staff_range'] = pd.to_numeric(test_df['ul_staff_range'])

Посчитаем столбцы с высокой корреляцией  

In [9]:
corr_matrix = train_df.corr().abs()
high_corr_var=np.where(corr_matrix>0.9)
high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
high_corr_var

[('ar_revenue', 'ar_total_expenses'),
 ('ar_revenue', 'ar_sale_cost'),
 ('ar_total_expenses', 'ar_sale_cost'),
 ('ar_sale_profit', 'ar_profit_before_tax'),
 ('ar_sale_profit', 'ar_net_profit'),
 ('ar_profit_before_tax', 'ar_net_profit'),
 ('adr_actual_age', 'head_actual_age'),
 ('adr_actual_age', 'cap_actual_age'),
 ('head_actual_age', 'cap_actual_age')]

In [10]:
train_df['ul_systematizing_flg'].unique()

array([0])

Также видим что признак ul_systematizing_flg имеет одно значение для всех компаний, следовательно его нужно удалить, как и некоторые сильно коррелирующие столбцы 

In [11]:
train_df.drop(['ul_systematizing_flg', 'ul_strategic_flg', 'ar_total_expenses', 'ar_profit_before_tax', 'ar_net_profit',  'ar_sale_cost', 'head_actual_age', 'cap_actual_age','record_id' ], axis='columns', inplace=True)
test_df.drop(['ul_systematizing_flg', 'ul_strategic_flg', 'ar_total_expenses', 'ar_profit_before_tax', 'ar_net_profit', 'ar_sale_cost','head_actual_age', 'cap_actual_age','record_id' ], axis='columns', inplace=True)

In [12]:
train_df.isnull().sum(axis = 0)

ar_revenue                      15145
ar_selling_expenses             15145
ar_management_expenses          15145
ar_sale_profit                  15145
ar_balance_of_rvns_and_expns    15145
ar_taxes                        15145
ar_other_profit_and_losses      15145
ab_immobilized_assets           15145
ab_mobile_current_assets        15145
ab_inventory                    15145
ab_accounts_receivable          15145
ab_other_current_assets         15145
ab_cash_and_securities          15145
ab_losses                       15145
ab_own_capital                  15145
ab_borrowed_capital             15145
ab_long_term_liabilities        15145
ab_short_term_borrowing         15145
ab_accounts_payable             15145
ab_other_borrowings             15145
bus_age                             0
ogrn_age                            0
adr_actual_age                      0
ul_staff_range                      0
ul_capital_sum                      0
ul_founders_cnt                     0
ul_branch_cn

Видим, что слишком много пропущенных значений в фин отчетности в обучающих данных и заполнить их каким либо образом не представляется возможным, следовательно имеет место построение 2 моделей, одну для компаний с фин отчетностью, другую - для компаний без

Будем обозначать посфиксом first - компании без фин отчетности и second компании с ней

In [14]:
df= train_df.dropna() 

In [15]:
delete = ['ar_revenue',
 'ar_selling_expenses',
 'ar_management_expenses',
 'ar_sale_profit',
 'ar_balance_of_rvns_and_expns',
 'ar_taxes',
 'ar_other_profit_and_losses',
 'ab_immobilized_assets',
 'ab_mobile_current_assets',
 'ab_inventory',
 'ab_accounts_receivable',
 'ab_other_current_assets',
 'ab_cash_and_securities',
 'ab_losses',
 'ab_own_capital',
 'ab_borrowed_capital',
 'ab_long_term_liabilities',
 'ab_short_term_borrowing',
 'ab_accounts_payable',
 'ab_other_borrowings']

In [16]:
test_df_second = test_df.dropna() 

In [17]:
df_first = train_df.loc[~train_df.index.isin(df.index)]
df_first.drop(delete, axis='columns', inplace=True)
test_df_first = test_df.loc[~test_df.index.isin(test_df_second.index)]
test_df_first.drop(delete, axis='columns', inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Вместо генерации новых фичей возьмем логарифм от всех величин

In [18]:
X_first=df_first.drop(['default_12m'],axis=1)
for i in X_first.columns:
  mn = min(df_first[i].min(), test_df_first[i].min()) - 1
  df_first[i] = np.log(df_first[i]- mn)
  test_df_first[i] = np.log(test_df_first[i]- mn)
X_first=df_first.drop(['default_12m' ],axis=1)
Y_first=df_first['default_12m']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [20]:
X_second=df.drop(['default_12m' ],axis=1) 
for i in X_second.columns:
  mn = min(df[i].min(), test_df_second[i].min()) - 1
  df[i] = np.log(df[i]- mn)
  test_df_second[i] = np.log(test_df_second[i]- mn)
X_second=df.drop(['default_12m' ],axis=1) 
Y_second=df['default_12m']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Гиперпараметры модели подобраны опытным путем

In [22]:
model_first = CatBoostRegressor(depth = 12, l2_leaf_reg = 4, iterations = 1200, learning_rate = 0.08)
model_first.fit(X_first, Y_first, verbose = False)
model_second = CatBoostRegressor(depth = 10, l2_leaf_reg = 2, iterations = 1200, learning_rate = 0.07)
model_second.fit(X_second, Y_second, verbose = False)


In [23]:
cv_baseline_first = cross_val_score(model_first, X_first, Y_first, cv=5, scoring='roc_auc').mean()

Выходные данные были обрезаны до нескольких последних строк (5000).
1000:	learn: 0.2179531	total: 43.1s	remaining: 8.57s
1001:	learn: 0.2179389	total: 43.1s	remaining: 8.52s
1002:	learn: 0.2179239	total: 43.2s	remaining: 8.48s
1003:	learn: 0.2179056	total: 43.2s	remaining: 8.44s
1004:	learn: 0.2178971	total: 43.3s	remaining: 8.4s
1005:	learn: 0.2178261	total: 43.3s	remaining: 8.35s
1006:	learn: 0.2178095	total: 43.4s	remaining: 8.31s
1007:	learn: 0.2177899	total: 43.4s	remaining: 8.27s
1008:	learn: 0.2177592	total: 43.5s	remaining: 8.22s
1009:	learn: 0.2177517	total: 43.5s	remaining: 8.18s
1010:	learn: 0.2176627	total: 43.5s	remaining: 8.14s
1011:	learn: 0.2176423	total: 43.6s	remaining: 8.1s
1012:	learn: 0.2176055	total: 43.6s	remaining: 8.05s
1013:	learn: 0.2176002	total: 43.7s	remaining: 8.01s
1014:	learn: 0.2175892	total: 43.7s	remaining: 7.97s
1015:	learn: 0.2175469	total: 43.8s	remaining: 7.92s
1016:	learn: 0.2175298	total: 43.8s	remaining: 7.88s
1017:	learn: 0.2174735	total: 43.

In [24]:
cv_baseline_second = cross_val_score(model_second, X_second, Y_second, cv=3, scoring='roc_auc').mean()

0:	learn: 0.2072008	total: 67.9ms	remaining: 1m 21s
1:	learn: 0.2062787	total: 139ms	remaining: 1m 23s
2:	learn: 0.2058144	total: 217ms	remaining: 1m 26s
3:	learn: 0.2048615	total: 288ms	remaining: 1m 26s
4:	learn: 0.2042701	total: 360ms	remaining: 1m 26s
5:	learn: 0.2037064	total: 428ms	remaining: 1m 25s
6:	learn: 0.2031714	total: 504ms	remaining: 1m 25s
7:	learn: 0.2024960	total: 576ms	remaining: 1m 25s
8:	learn: 0.2019058	total: 647ms	remaining: 1m 25s
9:	learn: 0.2013130	total: 717ms	remaining: 1m 25s
10:	learn: 0.2008051	total: 796ms	remaining: 1m 26s
11:	learn: 0.2003793	total: 866ms	remaining: 1m 25s
12:	learn: 0.1999147	total: 936ms	remaining: 1m 25s
13:	learn: 0.1993989	total: 1s	remaining: 1m 24s
14:	learn: 0.1987359	total: 1.07s	remaining: 1m 24s
15:	learn: 0.1984285	total: 1.14s	remaining: 1m 24s
16:	learn: 0.1980077	total: 1.22s	remaining: 1m 24s
17:	learn: 0.1976537	total: 1.29s	remaining: 1m 24s
18:	learn: 0.1970440	total: 1.37s	remaining: 1m 25s
19:	learn: 0.1964217	tot

Выведем ROC/AUC моделей

In [25]:
print('ROC/AUC моделей : ', cv_baseline_first, cv_baseline_second)

ROC/AUC моделей :  0.6296983180315971 0.7217224049312053


Создадим предикты и преобразуем их в файл ответа 

In [26]:
cutoff_first = model_first.predict(test_df_first)
cutoff_first = sorted(cutoff_first)[54]
predict_first  = (model_first.predict(test_df_first)>= cutoff_first).astype(int)

cutoff_second = model_second.predict(test_df_second)
cutoff_second = sorted(cutoff_second)[45]
predict_second  = (model_second.predict(test_df_second) >= cutoff_second).astype(int)

test_df_second['predict'] = predict_second
test_df_first['predict'] = predict_first
answ_df =pd.concat([test_df_second['predict'], test_df_first['predict']])
answ_df.index.name = 'id'
answ_df.sort_index(inplace=True)
answ_df.to_csv('PD-submit.csv',index=True, sep=';')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
